<a href="https://colab.research.google.com/github/yukinaga/ai_webapp/blob/main/section_3/01_image_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 画像認識アプリ
Streamlitを使い、画像を認識するアプリを作りましょう。  
フレームワークにはPyTorchを、モデルにはResNetを使用します。

## ●ライブラリのインストール
Streamlit、およびアプリの動作の確認に使用する「ngrok」をインストールします。

In [ ]:
!pip install streamlit==1.7.0 --quiet
!pip install pyngrok==4.1.1 --quiet

Streamlit、およびngrokをインポートしておきます。  
エラーが発生する場合は、「ランタイム」→「ランタイムを再起動」によりランタイムを再起動し、再びコードセルを上から順に実行しましょう。

In [ ]:
import streamlit as st
from pyngrok import ngrok

## ●「モデル」を扱うファイル
画像認識のモデル「ResNet」を読み込み、予測を行うコードを「model.py」に書き込みます。  
今回は、訓練済みのResNet-101を読み込みます。  
ResNet-101は、深さが101層の畳み込みニューラル ネットワークです。  
https://pytorch.org/vision/stable/generated/torchvision.models.resnet101.html  
分類の各ラベルは「imagenet_classes.txt」に格納されているので、アップロードしておきましょう。

In [ ]:
%%writefile model.py
# 以下を「model.py」に書き込み
import torch
from torchvision import models, transforms
from PIL import Image

net = models.resnet101(pretrained=True)  # 訓練済みのモデルを読み込み
with open("imagenet_classes.txt") as f:  # ラベルの読み込み
    classes = [line.strip() for line in f.readlines()]

def predict(img):
    # 以下の設定はこちらを参考に設定: https://pytorch.org/hub/pytorch_vision_resnet/
    transform = transforms.Compose([
                                    transforms.Resize(256),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize(
                                        mean=[0.485, 0.456, 0.406],
                                        std=[0.229, 0.224, 0.225]
                                        )
                                    ])

    # モデルへの入力
    img = transform(img)
    x = torch.unsqueeze(img, 0)  # バッチ対応

    # 予測
    net.eval()
    y = net(x)

    # 結果を返す
    y_prob = torch.nn.functional.softmax(torch.squeeze(y))  # 確率で表す
    sorted_prob, sorted_indices = torch.sort(y_prob, descending=True)  # 降順にソート
    return [(classes[idx], prob.item()) for idx, prob in zip(sorted_indices, sorted_prob)]

## ●アプリのコード
画像認識アプリのコードを、「app.py」に書き込みます。  
ローカルからアップロード、もしくはWebカメラで撮影した画像ファイルに、何が映っているのかを判定します。  
なお、Webカメラはngrokが発行したURLではセキュリティ上動作しないので、今回は動作を確認できません。

In [ ]:
%%writefile app.py
# 以下を「app.py」に書き込み
import streamlit as st
import matplotlib.pyplot as plt
from PIL import Image
from model import predict

st.set_option("deprecation.showfileUploaderEncoding", False)

st.sidebar.title("画像認識アプリ")
st.sidebar.write("ResNetを使って何の画像かを判定します。")

st.sidebar.write("")

img_source = st.sidebar.radio("画像のソースを選択してください。",
                              ("画像をアップロード", "カメラで撮影"))
if img_source == "画像をアップロード":
    img_file = st.sidebar.file_uploader("画像を選択してください。", type=["png", "jpg"])
elif img_source == "カメラで撮影":
    img_file = st.camera_input("カメラで撮影")

if img_file is not None:
    with st.spinner("推定中..."):
        img = Image.open(img_file)
        st.image(img, caption="対象の画像", width=480)
        st.write("")

        # 予測
        results = predict(img)

        # 結果の表示
        st.subheader("判定結果")
        n_top = 5  # 確率が高い順に5位まで返す
        for result in results[:n_top]:
            st.write(str(round(result[1]*100, 2)) + "%の確率で" + result[0] + "です。")

        # 円グラフの表示
        pie_labels = [result[0] for result in results[:n_top]]
        pie_labels.append("others")
        pie_probs = [result[1] for result in results[:n_top]]
        pie_probs.append(sum([result[1] for result in results[n_top:]]))
        fig, ax = plt.subplots()
        wedgeprops={"width":0.3, "edgecolor":'white'}
        textprops = {"fontsize":6}
        ax.pie(pie_probs, labels=pie_labels, counterclock=False, startangle=90,
               textprops=textprops, autopct="%.2f", wedgeprops=wedgeprops)  # 円グラフ
        st.pyplot(fig)

## ●Authtokenの設定
ngrokで接続するために必要な「Authtoken」を設定します。  
以下のコードの、  
`!ngrok authtoken YourAuthtoken`  
における  
`YourAuthtoken`の箇所を、自分のAuthtokenに置き換えます。  
Authtokenは、ngrokのサイトに登録すれば取得することができます。  
https://ngrok.com/


In [ ]:
!ngrok authtoken YourAuthtoken

## ●アプリの起動と動作確認
streamlitの`run`コマンドでアプリを起動します。


In [ ]:
!streamlit run app.py &>/dev/null&  # 「&>/dev/null&」により、出力を非表示にしてバックグランドジョブとして実行

ngrokのプロセスを終了した上で、新たにポートを指定して接続します。  
接続の結果、urlを取得できます。  
ngrokの無料プランでは同時に1つのプロセスしか動かせないので、エラーが発生した場合は「ランタイム」→「セッションの管理」で不要なGoogle Colabのセッションを修了しましょう。  

In [ ]:
ngrok.kill()  # プロセスの修了
url = ngrok.connect(port="8501")  # 接続

urlを表示し、リンク先でアプリが動作することを確認します。

In [ ]:
print(url)